In [ ]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install tqdm
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
import re
import logging
from datetime import datetime

import torch
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS as SPACY_STOP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from typing import List, Dict, Any

In [ ]:
INPUT_PATH = "/content/drive/MyDrive/Лента/all.xlsx"
OUTPUT_DIR = "/content/drive/MyDrive/Лента/results"
CONFIDENCE_THRESHOLD = 0.5
MAX_LENGTH = 300
BATCH_SIZE = 32
MODEL_NAME = "facebook/bart-large-mnli"

# Настройка логирования
os.makedirs(OUTPUT_DIR, exist_ok=True)
log_file = os.path.join(OUTPUT_DIR, 'infopovod_analyzer.log')
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

CATEGORIES = {
    "discoveries": [
        "открытие нового вида животных в Сибири",
        "обнаружение экзопланеты в зоне обитаемости",
        "подтверждение существования гравитационных волн",
        "синтез сверхпроводника при комнатной температуре",
        "выявление механизма регенерации нейронов",
        "обнаружение древней цивилизации археологами",
        "открытие нового свойства графена",
        "доказательство гипотезы Римана",
        "идентификация генов долголетия",
        "наблюдение квантовой запутанности в макромире"
    ],

    "technology_innovation": [
        "разработан прототип квантового процессора",
        "учёные создали новый тип бионического глаза",
        "изобретён аккумулятор с рекордной ёмкостью",
        "предложен революционный алгоритм машинного обучения",
        "разработана методика 3D-печати человеческих органов",
        "открыт новый принцип работы нанороботов",
        "патент на уникальный нейроинтерфейс",
        "экспериментальная технология синтеза графена",
        "теоретическое обоснование квантовой телепортации",
        "программный код для моделирования ДНК"
    ],

    "funding": [
        "выделение 500 млн руб. на исследования Арктики",
        "стартап привлек $10 млн венчурных инвестиций",
        "ЕС одобрил грант в 2 млн евро на изучение климата",
        "РНФ поддержал 50 новых проектов",
        "корпорация инвестировала в биотех-лабораторию",
        "краудфандинг собрал 5 млн на космический спутник",
        "мегагрант на создание центра геномных технологий",
        "фонд Сороса выделил гранты молодым ученым",
        "бюджетное финансирование нацпроекта Наука",
        "частный инвестор вложился в термоядерный стартап"
    ],

    "partnership": [
        "МГУ и MIT создали совместную лабораторию",
        "подписание соглашения о научном сотрудничестве РФ и КНР",
        "консорциум пяти университетов по изучению AI",
        "фармкомпания и НИИ запустили программу разработки лекарств",
        "международный проект по картографированию океана",
        "создание альянса по борьбе с антибиотикорезистентностью",
        "российско-европейский проект в области квантовых технологий",
        "сотрудничество Роскосмоса и университетов в спутникостроении",
        "открытие совместного центра с CERN",
        "партнерство Tech-гиганта с академическими институтами"
    ],

    "regulation": [
        "принятие закона о редактировании генома человека",
        "введение новых стандартов клинических испытаний",
        "одобрение FDA нового класса препаратов",
        "запрет исследований в области евгеники",
        "регулирование использования ИИ в медицине",
        "лицензирование частных космических исследований",
        "введение этических норм для нейротехнологий",
        "новые правила обращения с радиоактивными материалами",
        "международная конвенция о клонировании",
        "сертификация первой российской вакцины ВОЗ"
    ],

    "project_launch": [
        "открыт завод по производству квантовых чипов",
        "начато серийное производство бионических протезов",
        "запущена линия аккумуляторов нового поколения",
        "внедрена система ИИ-диагностики в больницах",
        "стартовали клинические испытания вакцины",
        "введён в эксплуатацию новый синхротрон",
        "начал работу центр квантовых вычислений",
        "развёрнута сеть датчиков мониторинга климата",
        "марсоход начал исследование поверхности",
        "запущена национальная генетическая база данных"
    ],

    "events": [
        "Нобелевская неделя в Стокгольме",
        "Международный конгресс математиков",
        "Всероссийский фестиваль науки",
        "хакатон по квантовому машинному обучению",
        "Гайдаровский форум с участием нобелевских лауреатов",
        "презентация доклада по изменению климата",
        "выставка Технопром в Новосибирске",
        "съезд Российского союза ректоров",
        "конференция по биоэтике в Женеве",
        "Дни науки в Сколтехе с мастер-классами"
    ],

    "awards": [
        "вручение Нобелевской премии по химии 2024",
        "ученый получил Филдсовскую премию",
        "премия Глобальная энергия за термоядерные исследования",
        "награждение лауреатов премии Президента РФ",
        "медаль Дирака за вклад в теоретическую физику",
        "премия Тьюринга за прорыв в алгоритмах",
        "ученый удостоен звания Почетный профессор МГУ",
        "вручение государственной премии за нанотехнологии",
        "международная премия за исследования в области экологии",
        "награда Изобретатель года за медицинский прибор"
    ],

    "staff": [
        "назначение нового министра науки и высшего образования",
        "избрание президента РАН на новый срок",
        "смена ректора в ведущем техническом вузе",
        "переход нобелевского лауреата в российский университет",
        "назначение научного руководителя мегагранта",
        "создание нового департамента по квантовым технологиям",
        "увольнение декана факультета за нарушения",
        "приглашение ведущего зарубежного профессора",
        "реорганизация института с сокращением штата",
        "назначение молодого ученого заведующим лабораторией"
    ],

    "incidents": [
        "взрыв в химической лаборатории университета",
        "утечка радиоактивных материалов при транспортировке",
        "кибератака на базу данных исследований COVID",
        "кража экспериментальных образцов из лаборатории",
        "авария на ускорителе частиц с человеческими жертвами",
        "уничтожение уникальных образцов при отключении электричества",
        "заражение персонала опасным патогеном",
        "пожар в архиве с научными работами",
        "прорыв канализации в виварии",
        "обрушение конструкций строящегося научного центра"
    ],

    "controversies": [
        "отзыв статьи из Nature из-за фальсификаций",
        "скандал с плагиатом в диссертации чиновника",
        "конфликт за авторство открытия",
        "утечка данных незавершенного клинического испытания",
        "обвинения в жестоком обращении с лабораторными животными",
        "манипуляции с результатами фармацевтических испытаний",
        "коррупция при распределении мегагрантов",
        "дискриминация женщин-ученых в исследовательском центре",
        "подавление неудобных научных результатов",
        "использование студентов как бесплатной рабочей силы"
    ],

    "migration": [
        "массовый отъезд ученых после введения санкций",
        "программа возвращения российских ученых из-за рубежа",
        "переезд лаборатории в другую страну из-за финансирования",
        "история ученого, получившего позицию в Гарварде",
        "немецкий профессор возглавил кафедру в России",
        "утечка мозгов в биотехнологической отрасли",
        "российские постдоки в CERN",
        "закрытие совместных программ с зарубежными вузами",
        "трудоустройство беженцев-ученых в российских НИИ"
    ],

    "pandemic": [
        "разработка универсальной коронавирусной вакцины",
        "обнаружение нового опасного штамма гриппа",
        "исследования поствакцинального иммунитета",
        "создание препарата, сокращающего течение COVID-19",
        "разработка экспресс-теста с точностью 99%",
        "изучение долгосрочных последствий вируса",
        "пандемия привела к сокращению научного бюджета",
        "использование ИИ для прогнозирования вспышек",
        "международный проект по происхождению вируса",
        "нарушение цепочек поставок для исследований"
    ],

    "sanctions и сво": [
        "запрет поставок электроники для российских НИИ",
        "разработка отечественного аналога масс-спектрометра",
        "исключение университетов из международных коллабораций",
        "заморозка совместных проектов с западными партнерами",
        "создание фонда поддержки санкционных ученых",
        "переход на китайское лабораторное оборудование",
        "программа импортозамещения реагентов для ПЦР",
        "ограничения на публикации в зарубежных журналах",
        "национализация иностранных научных активов",
        "развитие собственных научных издательств"
    ],

    "obituaries": [
        "кончина академика, создателя вакцины от COVID",
        "трагическая гибель экспедиции в Антарктиде",
        "память о нобелевском лауреате, умершем в 95 лет",
        "смерть молодого перспективного ученого в аварии",
        "прощание с создателем советской космической программы",
        "некролог выдающемуся математику современности",
        "траур в институте после смерти заведующего кафедрой",
        "ученый скончался после продолжительной болезни",
        "мемориальная лекция в честь умершего физика",
        "посмертное награждение за незавершенное исследование"
    ]
}

class TextPreprocessor:
    def __init__(self):
        self.clean_pattern = re.compile(r'[^\w\sа-яА-ЯёЁ-]')
        self.max_length = MAX_LENGTH

    def clean_text(self, text: str) -> str:
        """Минимальная очистка текста без стоп-слов"""
        if not isinstance(text, str):
            return ""

        # Быстрая очистка: оставляем только буквы, цифры и дефисы
        cleaned = self.clean_pattern.sub(' ', text.lower())
        # Удаляем множественные пробелы и обрезаем по длине
        return re.sub(r'\s+', ' ', cleaned).strip()[:self.max_length]

def classify_batch(
    texts: List[str],
    classifier,
    labels: List[str],
    threshold: float
) -> List[Dict[str, Any]]:
    """Эффективная классификация батчами"""
    results = []
    try:
        for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Классификация"):
            batch = texts[i:i+BATCH_SIZE]
            outputs = classifier(
                batch,
                labels,
                truncation=True,
                max_length=MAX_LENGTH,
                multi_label=True,
                batch_size=BATCH_SIZE
            )

            for j, out in enumerate(outputs):
                results.append({
                    'text': batch[j],
                    'predicted_category': out['labels'][0],
                    'confidence': out['scores'][0],
                    'low_confidence': out['scores'][0] < threshold,
                    'top2_category': out['labels'][1],
                    'top2_confidence': out['scores'][1],
                    'top3_category': out['labels'][2],
                    'top3_confidence': out['scores'][2]
                })
    except Exception as e:
        logger.error(f"Ошибка классификации: {e}")
    return results

def main():
    logger.info("Инфоповод")

    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        device = 0
        torch_dtype = torch.float16
    else:
        device = -1
        torch_dtype = torch.float32

    logger.info(f"Устройство: {'GPU' if device == 0 else 'CPU'}")

    logger.info("Загрузка модели классификации...")
    classifier = pipeline(
        "zero-shot-classification",
        model=MODEL_NAME,
        device=device,
        framework="pt",
        torch_dtype=torch_dtype
    )
    logger.info("Загрузка данных...")
    try:
        df = pd.read_excel(INPUT_PATH)
        texts = df["text"].astype(str).tolist()
        logger.info(f"Загружено {len(texts)} статей")
    except Exception as e:
        logger.exception(f"Ошибка загрузки данных: {e}")
        return

    logger.info("Минимальная очистка текстов...")
    processor = TextPreprocessor()
    cleaned_texts = [processor.clean_text(txt) for txt in tqdm(texts, desc="Очистка")]

    categories = list(CATEGORIES.keys())

    logger.info("Начало классификации...")
    results = classify_batch(cleaned_texts, classifier, categories, CONFIDENCE_THRESHOLD)

    logger.info("Формирование результатов...")
    res_df = pd.DataFrame(results)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_file = os.path.join(OUTPUT_DIR, f"classified_{ts}.xlsx")
    logger.info(f"Сохранение результатов в {out_file}...")

    try:
        with pd.ExcelWriter(out_file) as writer:
            res_df.to_excel(writer, sheet_name="Details", index=False)

            summary = pd.concat([
                res_df['predicted_category'].value_counts().rename('Top1'),
                res_df['top2_category'].value_counts().rename('Top2'),
                res_df['top3_category'].value_counts().rename('Top3')
            ], axis=1).fillna(0).sort_values('Top1', ascending=False)

            summary.to_excel(writer, sheet_name="Summary")
        logger.info("Результаты успешно сохранены")
    except Exception as e:
        logger.exception(f"Ошибка при сохранении: {e}")

    logger.info("=== Завершено ===")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Классификация: 100%|██████████| 676/676 [1:29:55<00:00,  7.98s/it]
